## Case Number from multiple channels

- For the **COVID-19** research project


- By Chen Luo, on Feb 24, 2020


- [Click me to download the dataset](https://drive.google.com/open?id=1spRik17KrEZLbhTh3Kkf_V25j6CJFRWMTKQhUQU9xKQ)

In [1]:
import re
import json
import time
import random
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup

pd.set_option('max_rows', 500)
pd.set_option('max_columns', 500)

In [2]:
writer = pd.ExcelWriter('./caseNum.xlsx')

---
### 1. [Chinese Center for Disease Control and Prevention](http://www.chinacdc.cn/)
---

- Official release

- Data are extracted from this [Dashboard](http://2019ncov.chinacdc.cn/2019-nCoV/index.html) (daily update)

- By executing the following code blocks, we can get daily epidemical data

- Fields & avatars

> `新增确诊`: `A1`, `新增疑似`: `B1`, `新增死亡`: `C1`
>
> `累计确诊`: `A2`, `累计疑似`: `B2`, `累计死亡`: `C3`

- Sheet name in Excel: `CDC_CN`

In [3]:
headers = dict()
headers['user-agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36'
url = 'http://2019ncov.chinacdc.cn/2019-nCoV/dist/jkzx.bundle.js'

In [4]:
wp = requests.get(url, headers=headers).content
soup = BeautifulSoup(wp, 'html.parser')
pattern = re.compile('proviceData:(.*?),dataName')
data = json.loads(pattern.search(str(soup)).group(1))
index_list = list()
for province in data:
    for idx in ['A1', 'B1', 'C1', 'A2', 'B2', 'C2']:
        index_list.append(province.strip()[:2] + '_' + idx)
# index_list

In [5]:
column_list = list()
start_date = datetime.datetime.strptime("2020-01-15", "%Y-%m-%d")
while start_date.strftime("%Y-%m-%d") != datetime.datetime.now().strftime("%Y-%m-%d"):
    start_date = start_date + datetime.timedelta(days=1)
    column_list.append(start_date.strftime("%m-%d").strip())
# column_list

In [8]:
dataframe1 = pd.DataFrame(index=index_list, columns=column_list)
for province in data:
    for daily in data[province]:
        dataframe1.loc[province[:2] + '_' + 'A1', datetime.datetime.strptime(daily['日期'].
                                                                             replace('月', '-').
                                                                             replace('日', ''), 
                                                                             "%m-%d").strftime("%m-%d")] = int(daily['新增确诊'])
        dataframe1.loc[province[:2] + '_' + 'B1', datetime.datetime.strptime(daily['日期'].
                                                                             replace('月', '-').
                                                                             replace('日', ''), 
                                                                             "%m-%d").strftime("%m-%d")] = int(daily['新增疑似'])
        dataframe1.loc[province[:2] + '_' + 'C1', datetime.datetime.strptime(daily['日期'].
                                                                             replace('月', '-').
                                                                             replace('日', ''), 
                                                                             "%m-%d").strftime("%m-%d")] = int(daily['新增死亡'])
        dataframe1.loc[province[:2] + '_' + 'A2', datetime.datetime.strptime(daily['日期'].
                                                                             replace('月', '-').
                                                                             replace('日', ''), 
                                                                             "%m-%d").strftime("%m-%d")] = int(daily['累计确诊'])
        dataframe1.loc[province[:2] + '_' + 'B2', datetime.datetime.strptime(daily['日期'].
                                                                             replace('月', '-').
                                                                             replace('日', ''), 
                                                                             "%m-%d").strftime("%m-%d")] = int(daily['累计疑似'])
        dataframe1.loc[province[:2] + '_' + 'C2', datetime.datetime.strptime(daily['日期'].
                                                                             replace('月', '-').
                                                                             replace('日', ''), 
                                                                             "%m-%d").strftime("%m-%d")] = int(daily['累计死亡'])
dataframe1.head(20)

,01-16,01-17,01-18,01-19,01-20,01-21,01-22,01-23,01-24,01-25,01-26,01-27,01-28,01-29,01-30,01-31,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,02-13,02-14,02-15,02-16,02-17,02-18,02-19,02-20,02-21,02-22,02-23,02-24,02-25
澳门_A1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
澳门_B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
澳门_C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
澳门_A2,0,0,0,0,0,0,0,0,0,0,0,7,7,7,7,7,7,8,8,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,NaN
澳门_B2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
澳门_C2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
香港_A1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,1,1,1,0,3,3,3,2,0,10,6,7,1,3,3,0,1,3,2,3,3,0,1,5,7,NaN
香港_B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
香港_C1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
香港_A2,0,0,0,0,0,0,0,0,0,0,0,8,8,10,12,13,14,15,15,18,21,24,26,26,36,42,49,50,53,56,56,57,60,62,65,68,68,69,74,81,NaN


In [9]:
dataframe1.to_excel(writer, sheet_name='CDC_CN', header=True, index=True)

---
### 2. [A crowdsourcing project on GitHub: Academic2019-nCoV](https://github.com/Academic-nCoV/2019-nCoV/wiki)
---

- Their data is consistent with the `CDC_CN` data

---
### 3. [Newly diagnosed nCoV statistics](https://github.com/839Studio/Novel-Coronavirus-Updates)
---

- Got the highest star on GitHub

- Daily update, supported by [Mei Shu Ke of the Paper](https://www.thepaper.cn/list_25635)

- Fine-grained data with source

- ⚠️Not comprehensive enough

- Sheet name in Excel: `839Studio`

In [10]:
dataframe2 = pd.read_csv('https://raw.githubusercontent.com/839Studio/Novel-Coronavirus-Updates/master/Updates_NC.csv', sep=',')
dataframe2.head(20)

,报道时间,省份,城市,新增确诊,新增出院,新增死亡,消息来源,来源链接1,来源链接2,来源链接3,备注
0,2月24日,重庆,沙坪坝区,1.0,0.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
1,2月24日,重庆,长寿区,1.0,0.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
2,2月24日,重庆,渝中区,0.0,1.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
3,2月24日,重庆,江北区,0.0,1.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
4,2月24日,重庆,江津区,0.0,1.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
5,2月24日,重庆,潼南区,0.0,1.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
6,2月24日,重庆,云阳县,0.0,2.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
7,2月24日,重庆,奉节县,0.0,1.0,0.0,重庆卫健委,http://wsjkw.cq.gov.cn/syyqzx/20200224/255967....,NaN,NaN,NaN
8,2月24日,意大利,NaN,20.0,0.0,1.0,央视,http://m.news.cctv.com/2020/02/24/ARTIPGpfx3sk...,NaN,NaN,NaN
9,2月24日,湖南,长沙市,0.0,2.0,0.0,湖南省卫健委,http://wjw.hunan.gov.cn/wjw/xxgk/gzdt/zyxw_1/2...,NaN,NaN,NaN


In [11]:
dataframe2.to_excel(writer, sheet_name='839Studio', header=True, index=None)

---
### 4. [Ding Xiang Yuan](https://3g.dxy.cn/newh5/view/pneumonia)
---

- Ding Xiang Yuan is the largest Chinese online community for physicians and health care professionals

- New field: `Cured`

- ⚠️Not uniform start date

- Fields & avatars

> `现存确诊`: `A1`, `现存新增确诊`: `A2`
>
> `新增确诊`: `B1`, `新增治愈`: `B2`, `新增死亡`: `B3`
>
> `累积确诊`: `C1`, `累计治愈`: `C2`, `累计死亡`: `C3`

- Sheet name in Excel: `DXY`

In [12]:
url_head = 'https://file1.dxycdn.com/2020/0223/'
province_dict = {
    '湖北': '618/3398299751673487511-135.json', 
    '广东': '281/3398299758115524068-135.json', 
    '浙江': '537/3398299755968455045-135.json', 
    '山东': '601/3398299749526003726-135.json', 
    '安徽': '734/3398299753820971301-135.json', 
    '江西': '161/3398299751673072165-135.json', 
    '河南': '958/3398299751673487456-135.json', 
    '湖南': '440/3398299751673072079-135.json', 
    '四川': '926/3398299755968455035-135.json', 
    '黑龙': '643/3398299753820971199-135.json', 
    '重庆': '368/3398299751673487486-135.json', 
    '北京': '004/3398299758115524169-135.json', 
    '江苏': '111/3398299753820971290-135.json', 
    '广西': '536/3398299758115523880-135.json', 
    '福建': '744/3398299751673071813-135.json', 
    '上海': '128/3398299755968454977-135.json', 
    '香港': '331/3398299755968040033-135.json', 
    '陕西': '857/3398299753820971228-135.json', 
    '河北': '473/3398299751673487477-135.json', 
    '新疆': '497/3398299753820556085-135.json', 
    '云南': '159/3398299758115523929-135.json', 
    '天津': '669/3398299753820555949-135.json', 
    '内蒙': '783/3398299758115938727-135.json', 
    '贵州': '148/3398299753820971243-135.json', 
    '海南': '126/3398299753820555862-135.json', 
    '辽宁': '815/3398299758115938736-135.json', 
    '山西': '196/3398299755968040081-135.json', 
    '吉林': '046/3398299755968039975-135.json', 
    '台湾': '045/3398299749526003760-135.json', 
    '宁夏': '353/3398299755968455019-135.json', 
    '甘肃': '559/3398299755968040166-135.json', 
    '澳门': '840/3398299753820971267-135.json', 
    '青海': '581/3398299758115524121-135.json', 
    '西藏': '353/3398299755968039885-135.json'
}

In [13]:
index_list = list()
for province in province_dict.keys():
    for idx in ['A1', 'A2', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']:
        index_list.append(province + '_' + idx)
# index_list

In [14]:
column_list = list()
start_date = datetime.datetime.strptime("2020-01-18", "%Y-%m-%d")
while start_date.strftime("%Y-%m-%d") != datetime.datetime.now().strftime("%Y-%m-%d"):
    start_date = start_date + datetime.timedelta(days=1)
    column_list.append(start_date.strftime("%m-%d").strip())
# column_list

In [15]:
dataframe3 = pd.DataFrame(index=index_list, columns=column_list)
for province in province_dict.keys():
    print('Process >', province, url_head + province_dict[province])
    cursor = 0
    date_archive = list()
    province_data = json.loads(requests.get(url_head + province_dict[province]).content)['data']
    for item in province_data:
        date_archive.append(datetime.datetime.strftime(datetime.datetime.strptime(str(item['dateId']), '%Y%m%d'), 
                                                       '%m-%d'))
    for date in date_archive:
        dataframe3.loc[province + '_A1', date] = int(province_data[cursor]['currentConfirmedCount'])
        dataframe3.loc[province + '_A2', date] = int(province_data[cursor]['currentConfirmedIncr'])
        dataframe3.loc[province + '_B1', date] = int(province_data[cursor]['confirmedIncr'])
        dataframe3.loc[province + '_B2', date] = int(province_data[cursor]['curedIncr'])
        dataframe3.loc[province + '_B3', date] = int(province_data[cursor]['deadIncr'])
        dataframe3.loc[province + '_C1', date] = int(province_data[cursor]['confirmedCount'])
        dataframe3.loc[province + '_C2', date] = int(province_data[cursor]['curedCount'])
        dataframe3.loc[province + '_C3', date] = int(province_data[cursor]['deadCount'])
        cursor += 1
    # take care of the server, please do not comment out the next line
    time.sleep(random.randint(5, 10))

Process > 湖北 https://file1.dxycdn.com/2020/0223/618/3398299751673487511-135.json
Process > 广东 https://file1.dxycdn.com/2020/0223/281/3398299758115524068-135.json
Process > 浙江 https://file1.dxycdn.com/2020/0223/537/3398299755968455045-135.json
Process > 山东 https://file1.dxycdn.com/2020/0223/601/3398299749526003726-135.json
Process > 安徽 https://file1.dxycdn.com/2020/0223/734/3398299753820971301-135.json
Process > 江西 https://file1.dxycdn.com/2020/0223/161/3398299751673072165-135.json
Process > 河南 https://file1.dxycdn.com/2020/0223/958/3398299751673487456-135.json
Process > 湖南 https://file1.dxycdn.com/2020/0223/440/3398299751673072079-135.json
Process > 四川 https://file1.dxycdn.com/2020/0223/926/3398299755968455035-135.json
Process > 黑龙 https://file1.dxycdn.com/2020/0223/643/3398299753820971199-135.json
Process > 重庆 https://file1.dxycdn.com/2020/0223/368/3398299751673487486-135.json
Process > 北京 https://file1.dxycdn.com/2020/0223/004/3398299758115524169-135.json
Process > 江苏 https://file1.d

In [16]:
dataframe3.head(20)

,01-19,01-20,01-21,01-22,01-23,01-24,01-25,01-26,01-27,01-28,01-29,01-30,01-31,02-01,02-02,02-03,02-04,02-05,02-06,02-07,02-08,02-09,02-10,02-11,02-12,02-13,02-14,02-15,02-16,02-17,02-18,02-19,02-20,02-21,02-22,02-23,02-24,02-25
湖北_A1,NaN,239,338,399,494,658,958,1303,2567,3349,4334,5486,6738,8565,10532,12712,15679,18483,20677,23139,24881,26965,28532,29659,43455,46429,48175,49030,49847,50338,50633,49665,48730,47647,46439,45054,43369,NaN
湖北_A2,NaN,239,99,61,95,164,300,345,1264,782,985,1152,1252,1827,1967,2180,2967,2804,2194,2462,1742,2084,1567,1127,13796,2974,1746,855,817,491,295,-968,-935,-1083,-1208,-1385,-1685,NaN
湖北_B1,NaN,270,105,69,105,180,323,371,1291,840,1032,1220,1347,1921,2103,2345,3156,2987,2447,2841,2147,2531,2097,1638,14840,3780,2420,1843,1933,1807,1693,349,631,792,630,203,499,NaN
湖北_B2,NaN,25,3,0,3,1,10,2,3,33,10,26,50,49,80,101,124,113,184,298,324,356,427,417,802,690,643,849,1016,1223,1266,1209,1451,1769,1742,1439,2116,NaN
湖北_B3,NaN,6,3,8,7,15,13,24,24,25,37,42,45,45,56,64,65,70,69,81,81,91,103,94,242,116,31,139,100,93,132,108,115,106,96,149,68,NaN
湖北_C1,NaN,270,375,444,549,729,1052,1423,2714,3554,4586,5806,7153,9074,11177,13522,16678,19665,22112,24953,27100,29631,31728,33366,48206,51986,54406,56249,58182,59989,61682,62031,62662,63454,64084,64287,64786,NaN
湖北_C2,NaN,25,28,28,31,32,42,44,47,80,90,116,166,215,295,396,520,633,817,1115,1439,1795,2222,2639,3441,4131,4774,5623,6639,7862,9128,10337,11788,13557,15299,16738,18854,NaN
湖北_C3,NaN,6,9,17,24,39,52,76,100,125,162,204,249,294,350,414,479,549,618,699,780,871,974,1068,1310,1426,1457,1596,1696,1789,1921,2029,2144,2250,2346,2495,2563,NaN
广东_A1,1,17,26,32,53,76,109,144,184,236,305,386,512,592,669,777,838,895,949,977,994,1007,995,977,955,927,906,878,845,794,755,708,664,614,596,567,535,NaN
广东_A2,1,16,9,6,21,23,33,35,40,52,69,81,126,80,77,108,61,57,54,28,17,13,-12,-18,-22,-28,-21,-28,-33,-51,-39,-47,-44,-50,-18,-29,-32,NaN


In [17]:
dataframe3.to_excel(writer, sheet_name='DXY', header=True, index=True)

---
### 5. [Tencent News](https://news.qq.com/zt2020/page/feiyan.htm)
---

- Same as Ding Xiang Yuan

---
### 6. [Sina News](https://news.sina.cn/zt_d/yiqing0121)
---

- Same as Ding Xiang Yuan

---
### 7. [Beijing Daily](https://bjrbh5.bjd.com.cn/web/page)
---

- Just provides the latest data

---
### 8. [Xinhua Net](http://fms.news.cn/swf/2020_sjxw/2_1_xgyq/)
---

- Same as Ding Xiang Yuan

In [18]:
writer.close()